In [ ]:
!pip install pandas BeautifulSoup4 selenium elasticsearch
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 968 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 54.2 MB/s 
     |████████████████████████████████| 359 kB 61.0 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 3.6 MB 55.5 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu

In [ ]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from bs4 import BeautifulSoup as bfs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import InvalidArgumentException, WebDriverException

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df

,title,author,date,views,likes,link,transcript
0,Climate action needs new frontline leadership,Ozawa Bineshi Albert,December 2021,404000,12000,https://ted.com/talks/ozawa_bineshi_albert_cli...,Yuchi F’as English Good afternoon I come from ...
1,The dark history of the overthrow of Hawaii,Sydney Iaukea,February 2022,214000,6400,https://ted.com/talks/sydney_iaukea_the_dark_h...,It was January 16th 1895 Two men arrived at Li...
2,How play can spark new ideas for your business,Martin Reeves,September 2021,412000,12000,https://ted.com/talks/martin_reeves_how_play_c...,Have you ever trodden on a Lego brick in your ...
3,Why is China appointing judges to combat clima...,James K. Thornton,October 2021,427000,12000,https://ted.com/talks/james_k_thornton_why_is_...,Imagine a world in which China was an environm...
4,Cement's carbon problem — and 2 ways to fix it,Mahendra Singhi,October 2021,2400,72,https://ted.com/talks/mahendra_singhi_cement_s...,NaN
...,...,...,...,...,...,...,...
5435,The best stats you've ever seen,Hans Rosling,February 2006,15000000,458000,https://ted.com/talks/hans_rosling_the_best_st...,About 10 years ago I took on the task to teach...
5436,Do schools kill creativity?,Sir Ken Robinson,February 2006,72000000,2100000,https://ted.com/talks/sir_ken_robinson_do_scho...,Good morning How are you Audience Good Its bee...
5437,Greening the ghetto,Majora Carter,February 2006,2900000,88000,https://ted.com/talks/majora_carter_greening_t...,If youre here today and Im very happy that you...
5438,Simplicity sells,David Pogue,February 2006,2000000,60000,https://ted.com/talks/david_pogue_simplicity_s...,Music The Sound of Silence Simon amp Garfunkel...


In [ ]:
# Initializing headless browser for selenium
browser_options = webdriver.ChromeOptions()
browser_options.add_argument('--headless')
browser_options.add_argument('--no-sandbox')
browser_options.add_argument('--disable-dev-shm-usage')
browser_options.add_argument("--lang=en-US")   # making sure we get english transcript
browser_service = Service('chromedriver')      # need to make chromdriver executable

In [ ]:
def get_transcript(index, url):
    url += '/transcript'
    browser = webdriver.Chrome(options=browser_options, service=browser_service)      
  
    try:
        browser.get(url)
    except InvalidArgumentException:
        browser.close()
        return index, None    # return None if request error 

    html = browser.page_source
    browser.close()
    bfs_html = bfs(html, features="html.parser")
    transcript_bfs_list = bfs_html.find_all('span', attrs={'class': 'cursor-pointer inline hover:bg-red-300 css-82uonn'})

    if transcript_bfs_list:
        transcript_list = [transcript.text.strip() for transcript in transcript_bfs_list]
    else:
        return index, None    # return None if transcript was not found

    return index, ' '.join(transcript_list)

In [ ]:
with ProcessPoolExecutor(max_workers=5) as executor:
    # getting rows that have transcript field set to Null
    input_data = df.loc[df['transcript'].isnull()].iterrows()
    
    results = [executor.submit(get_transcript, index, row['link']) for index, row in input_data]
    for result in as_completed(results):
      try:
        index, transcript = result.result()
        df.loc[index, 'transcript'] = transcript
        if transcript is not None: print(f'Completed: {index},\t Remaining: {len(df) - len(df.loc[~df["transcript"].isnull()])}')
      except WebDriverException:
        pass  # ignoring chromedriver errors

In [ ]:
df

,title,author,date,views,likes,link,transcript
0,Climate action needs new frontline leadership,Ozawa Bineshi Albert,December 2021,404000,12000,https://ted.com/talks/ozawa_bineshi_albert_cli...,Yuchi F’as English Good afternoon I come from ...
1,The dark history of the overthrow of Hawaii,Sydney Iaukea,February 2022,214000,6400,https://ted.com/talks/sydney_iaukea_the_dark_h...,It was January 16th 1895 Two men arrived at Li...
2,How play can spark new ideas for your business,Martin Reeves,September 2021,412000,12000,https://ted.com/talks/martin_reeves_how_play_c...,Have you ever trodden on a Lego brick in your ...
3,Why is China appointing judges to combat clima...,James K. Thornton,October 2021,427000,12000,https://ted.com/talks/james_k_thornton_why_is_...,Imagine a world in which China was an environm...
4,Cement's carbon problem — and 2 ways to fix it,Mahendra Singhi,October 2021,2400,72,https://ted.com/talks/mahendra_singhi_cement_s...,None
...,...,...,...,...,...,...,...
5435,The best stats you've ever seen,Hans Rosling,February 2006,15000000,458000,https://ted.com/talks/hans_rosling_the_best_st...,About 10 years ago I took on the task to teach...
5436,Do schools kill creativity?,Sir Ken Robinson,February 2006,72000000,2100000,https://ted.com/talks/sir_ken_robinson_do_scho...,Good morning How are you Audience Good Its bee...
5437,Greening the ghetto,Majora Carter,February 2006,2900000,88000,https://ted.com/talks/majora_carter_greening_t...,If youre here today and Im very happy that you...
5438,Simplicity sells,David Pogue,February 2006,2000000,60000,https://ted.com/talks/david_pogue_simplicity_s...,Music The Sound of Silence Simon amp Garfunkel...


In [ ]:
df.to_csv("data_with_transcript.csv", index=False)